In [2]:
import os
import numpy as np
import pandas as pd
home_folder = os.path.expanduser("/home/alvin/project/lab/Code_datamining")
# home_folder = sys.path
data_folder = os.path.join(home_folder, "Chapter 3", "NBA")
data_filename = os.path.join(data_folder, "leagues_NBA_2014_games_games.csv")
print(home_folder, data_filename)

/home/alvin/project/lab/Code_datamining /home/alvin/project/lab/Code_datamining/Chapter 3/NBA/leagues_NBA_2014_games_games.csv


In [3]:
results = pd.read_csv(data_filename)
results.loc[:5]

,Date,Unnamed: 1,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Notes
0,"Tue, Oct 29, 2013",Box Score,Orlando Magic,87,Indiana Pacers,97,NaN,NaN
1,"Tue, Oct 29, 2013",Box Score,Los Angeles Clippers,103,Los Angeles Lakers,116,NaN,NaN
2,"Tue, Oct 29, 2013",Box Score,Chicago Bulls,95,Miami Heat,107,NaN,NaN
3,"Wed, Oct 30, 2013",Box Score,Brooklyn Nets,94,Cleveland Cavaliers,98,NaN,NaN
4,"Wed, Oct 30, 2013",Box Score,Atlanta Hawks,109,Dallas Mavericks,118,NaN,NaN
5,"Wed, Oct 30, 2013",Box Score,Washington Wizards,102,Detroit Pistons,113,NaN,NaN


In [4]:
# Don't read the first row, as it is blank, and parse the date column as a date
# results = pd.read_csv(data_filename, parse_dates=['Date'], skiprows=[0,])  # 辣鸡！
results = pd.read_csv(data_filename, parse_dates=['Date'])  # 第二个参数表示以数字日期格式读取日期数据
# Fix the name of the columns 重命名各列的名称
results.columns = ['Date','Start (ET)','Visitor Team','VisitorPts','Home Team','HomePts','OT?',"Notes"]
# results.columns = ["Date", "Score Type", "Visitor Team", "VisitorPts", "Home Team", "HomePts", "OT?", "Notes"]
results.loc[:5]

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes
0,2013-10-29,Box Score,Orlando Magic,87,Indiana Pacers,97,NaN,NaN
1,2013-10-29,Box Score,Los Angeles Clippers,103,Los Angeles Lakers,116,NaN,NaN
2,2013-10-29,Box Score,Chicago Bulls,95,Miami Heat,107,NaN,NaN
3,2013-10-30,Box Score,Brooklyn Nets,94,Cleveland Cavaliers,98,NaN,NaN
4,2013-10-30,Box Score,Atlanta Hawks,109,Dallas Mavericks,118,NaN,NaN
5,2013-10-30,Box Score,Washington Wizards,102,Detroit Pistons,113,NaN,NaN


In [5]:
results["HomeWin"] = results["VisitorPts"] < results["HomePts"]  # 使用布尔判别语句得到队伍获胜情况（主场）
# Our "class values"
y_true = results["HomeWin"].values
print(results.loc[:5])
print('\n这个月数据规模{}'.format(results.shape))

        Date Start (ET)          Visitor Team  VisitorPts  \
0 2013-10-29  Box Score         Orlando Magic          87   
1 2013-10-29  Box Score  Los Angeles Clippers         103   
2 2013-10-29  Box Score         Chicago Bulls          95   
3 2013-10-30  Box Score         Brooklyn Nets          94   
4 2013-10-30  Box Score         Atlanta Hawks         109   
5 2013-10-30  Box Score    Washington Wizards         102   

             Home Team  HomePts  OT? Notes  HomeWin  
0       Indiana Pacers       97  NaN   NaN     True  
1   Los Angeles Lakers      116  NaN   NaN     True  
2           Miami Heat      107  NaN   NaN     True  
3  Cleveland Cavaliers       98  NaN   NaN     True  
4     Dallas Mavericks      118  NaN   NaN     True  
5      Detroit Pistons      113  NaN   NaN     True  

这个月数据规模(1319, 9)


In [6]:
print("Home Win percentage: {0:.1f}%".format(100 * results["HomeWin"].sum() / results["HomeWin"].count()))
# 猜测是基于字典的数据类型 （count方法）

Home Win percentage: 57.9%


In [7]:
'''
采用球队上两场比赛的胜负情况作为初始特征，新建两列（一次主场情况，一次客场情况）
'''
results["HomeLastWin"] = False
results["VisitorLastWin"] = False
# This creates two new columns, all set to False
results.iloc[:5]

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes,HomeWin,HomeLastWin,VisitorLastWin
0,2013-10-29,Box Score,Orlando Magic,87,Indiana Pacers,97,NaN,NaN,True,False,False
1,2013-10-29,Box Score,Los Angeles Clippers,103,Los Angeles Lakers,116,NaN,NaN,True,False,False
2,2013-10-29,Box Score,Chicago Bulls,95,Miami Heat,107,NaN,NaN,True,False,False
3,2013-10-30,Box Score,Brooklyn Nets,94,Cleveland Cavaliers,98,NaN,NaN,True,False,False
4,2013-10-30,Box Score,Atlanta Hawks,109,Dallas Mavericks,118,NaN,NaN,True,False,False


In [8]:
# Now compute the actual values for these
# Did the home and visitor teams win their last game?
from collections import defaultdict
won_last = defaultdict(int)  # 初始化字典类型为int
'''
以下代码假设数据按照时间顺序排列，如果数据乱序则需要按日期排列
results.iterrows()要被替换为results.sort('Date').iterrows() # 按照日期排序
'''
for index, row in results.iterrows():  # Note that this is not efficient
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["HomeLastWin"] = won_last[home_team]
    row["VisitorLastWin"] = won_last[visitor_team]
    results.loc[index] = row    
    # Set current win
    # 遍历数据，每次都进行主队客队胜负信息的更新     
    won_last[home_team] = row["HomeWin"]
    won_last[visitor_team] = not row["HomeWin"]
results.iloc[20:25]

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes,HomeWin,HomeLastWin,VisitorLastWin
20,2013-11-01,Box Score,Milwaukee Bucks,105,Boston Celtics,98,NaN,NaN,False,False,False
21,2013-11-01,Box Score,Miami Heat,100,Brooklyn Nets,101,NaN,NaN,True,False,False
22,2013-11-01,Box Score,Cleveland Cavaliers,84,Charlotte Bobcats,90,NaN,NaN,True,False,True
23,2013-11-01,Box Score,Portland Trail Blazers,113,Denver Nuggets,98,NaN,NaN,False,False,False
24,2013-11-01,Box Score,Dallas Mavericks,105,Houston Rockets,113,NaN,NaN,True,True,True


In [9]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=14)

In [10]:
from sklearn.model_selection import cross_val_score

# Create a dataset with just the neccessary information
X_previouswins = results[["HomeLastWin", "VisitorLastWin"]].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_previouswins, y_true, scoring='accuracy')
print("Using just the last result from the home and visitor teams")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Using just the last result from the home and visitor teams
Accuracy: 57.5%


In [12]:
# What about win streaks?
results["HomeWinStreak"] = 0
results["VisitorWinStreak"] = 0
# Did the home and visitor teams win their last game?
from collections import defaultdict
win_streak = defaultdict(int)

for index, row in results.iterrows():  # Note that this is not efficient
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["HomeWinStreak"] = win_streak[home_team]
    row["VisitorWinStreak"] = win_streak[visitor_team]
    results.ix[index] = row    
    # Set current win  统计某队伍主场客场赢得比赛的次数
    if row["HomeWin"]:
        win_streak[home_team] += 1
        win_streak[visitor_team] = 0
    else:
        win_streak[home_team] = 0
        win_streak[visitor_team] += 1
results.iloc[120:125]

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes,HomeWin,HomeLastWin,VisitorLastWin,HomeWinStreak,VisitorWinStreak
120,2013-11-13,Box Score,Washington Wizards,79,San Antonio Spurs,92,NaN,NaN,True,True,False,5,0
121,2013-11-13,Box Score,New Orleans Pelicans,105,Utah Jazz,111,NaN,NaN,True,False,False,0,0
122,2013-11-14,Box Score,Oklahoma City Thunder,115,Golden State Warriors,116,NaN,NaN,True,True,False,1,0
123,2013-11-14,Box Score,Houston Rockets,109,New York Knicks,106,NaN,NaN,False,True,False,1,0
124,2013-11-15,Box Score,Philadelphia 76ers,103,Atlanta Hawks,113,NaN,NaN,True,False,True,0,1


In [13]:
clf = DecisionTreeClassifier(random_state=14)
X_winstreak =  results[["HomeLastWin", "VisitorLastWin", "HomeWinStreak", "VisitorWinStreak"]].values
scores = cross_val_score(clf, X_winstreak, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Using whether the home team is ranked higher
Accuracy: 57.1%


In [14]:
# Let's try see which team is better on the ladder. Using the previous year's ladder
# 使用上一年数据对今年的比赛做出预测
ladder_filename = os.path.join(data_folder, "leagues_NBA_2013_standings_expanded-standings.csv")
ladder = pd.read_csv(ladder_filename)
ladder

,Rk,Team,Overall,Home,Road,E,W,A,C,SE,...,Post,≤3,≥10,Oct,Nov,Dec,Jan,Feb,Mar,Apr
0,1,Miami Heat,66-16,37-4,29-12,41-11,25-5,14-4,12-6,15-1,...,30-2,9-3,39-8,1-0,10-3,10-5,8-5,12-1,17-1,8-1
1,2,Oklahoma City Thunder,60-22,34-7,26-15,21-9,39-13,7-3,8-2,6-4,...,21-8,3-6,44-6,NaN,13-4,11-2,11-5,7-4,12-5,6-2
2,3,San Antonio Spurs,58-24,35-6,23-18,25-5,33-19,8-2,9-1,8-2,...,16-12,9-5,31-10,1-0,12-4,12-4,12-3,8-3,10-4,3-6
3,4,Denver Nuggets,57-25,38-3,19-22,19-11,38-14,5-5,10-0,4-6,...,24-4,11-7,28-8,0-1,8-8,9-6,12-3,8-4,13-2,7-1
4,5,Los Angeles Clippers,56-26,32-9,24-17,21-9,35-17,7-3,8-2,6-4,...,17-9,3-5,38-12,1-0,8-6,16-0,9-7,8-5,7-7,7-1
5,6,Memphis Grizzlies,56-26,32-9,24-17,22-8,34-18,8-2,8-2,6-4,...,23-8,6-4,28-9,0-1,12-1,7-7,10-7,9-2,11-6,7-2
6,7,New York Knicks,54-28,31-10,23-18,37-15,17-13,10-6,12-6,15-3,...,22-10,7-5,31-12,NaN,11-4,10-5,7-6,6-5,12-6,8-2
7,8,Brooklyn Nets,49-33,26-15,23-18,36-16,13-17,11-5,13-5,12-6,...,18-11,9-4,23-17,NaN,11-4,5-11,11-4,7-5,8-7,7-2
8,9,Indiana Pacers,49-32,30-11,19-21,31-20,18-12,6-11,13-3,12-6,...,17-11,4-9,27-14,1-0,7-8,10-5,9-6,9-3,11-5,2-5
9,10,Golden State Warriors,47-35,28-13,19-22,19-11,28-24,7-3,5-5,7-3,...,17-13,5-3,20-18,1-0,8-6,12-4,8-7,4-8,9-7,5-3


In [15]:
# We can create a new feature -- HomeTeamRanksHigher\  建立一个新特征 即主场队是否通常比对手水平高
results["HomeTeamRanksHigher"] = 0
for index, row in results.iterrows():  # 遍历比赛记录
    # 当前客队主队     
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    if home_team == "New Orleans Pelicans":  # 鹈鹕队
        home_team = "New Orleans Hornets"  # 黄蜂队
    elif visitor_team == "New Orleans Pelicans":
        visitor_team = "New Orleans Hornets"
    home_rank = ladder[ladder["Team"] == home_team]["Rk"].values[0]  # 找出比赛记录中的主队的排名
    visitor_rank = ladder[ladder["Team"] == visitor_team]["Rk"].values[0]  # 找出比赛记录的客队的排名
    row["HomeTeamRanksHigher"] = int(home_rank < visitor_rank)  # 如果主队排名好于客队则特征置0
    results.ix[index] = row
results[:5]

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes,HomeWin,HomeLastWin,VisitorLastWin,HomeWinStreak,VisitorWinStreak,HomeTeamRanksHigher
0,2013-10-29,Box Score,Orlando Magic,87,Indiana Pacers,97,NaN,NaN,True,0,0,0,0,1
1,2013-10-29,Box Score,Los Angeles Clippers,103,Los Angeles Lakers,116,NaN,NaN,True,0,0,0,0,0
2,2013-10-29,Box Score,Chicago Bulls,95,Miami Heat,107,NaN,NaN,True,0,0,0,0,1
3,2013-10-30,Box Score,Brooklyn Nets,94,Cleveland Cavaliers,98,NaN,NaN,True,0,0,0,0,0
4,2013-10-30,Box Score,Atlanta Hawks,109,Dallas Mavericks,118,NaN,NaN,True,0,0,0,0,0


In [16]:
X_homehigher =  results[["HomeLastWin", "VisitorLastWin", "HomeTeamRanksHigher"]].values
print(X_homehigher)
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_homehigher, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

[[0 0 1]
 [0 0 0]
 [0 0 1]
 ...
 [True False 1]
 [False True 1]
 [True False 0]]
Using whether the home team is ranked higher
Accuracy: 59.7%


In [18]:
from sklearn.grid_search import GridSearchCV
# 网格搜索技术寻找决策树的最优参数， 下面参数为树的最大深度
parameter_space = {
                   "max_depth": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                   }
clf = DecisionTreeClassifier(random_state=14)
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_homehigher, y_true)
best_parameters = grid.best_best_parameters = grid.best_estimator_.get_params()
print("Accuracy: {0:.1f}%".format(grid.best_score_ * 100))
print('best_parameters:{}'.format(best_parameters))

Accuracy: 59.7%
best_parameters:{'min_weight_fraction_leaf': 0.0, 'min_samples_split': 2, 'max_depth': 2, 'presort': False, 'random_state': 14, 'max_features': None, 'min_samples_leaf': 1, 'class_weight': None, 'min_impurity_split': None, 'criterion': 'gini', 'splitter': 'best', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0}


In [83]:
# Who won the last match? We ignore home/visitor for this bit  新建特征，忽略主场客场因素，看上次两只球队之间的胜者是谁
last_match_winner = defaultdict(int)  # 字典默认的value为0 key为一个二元tuple,包含一场比赛的两支球队的信息
results["HomeTeamWonLast"] = 0  # 对于第一条数据默认其之前比赛没有获胜

for index, row in results.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    # 按照英文字母顺序表对球队名字进行排序,主队客队不分次序
    teams = tuple(sorted([home_team, visitor_team]))  # Sort for a consistent ordering  每次迭代teams都是一个二元tuple
    # Set in the row, who won the last encounter
    if last_match_winner[teams]:
        print(last_match_winner[teams])
#     print(teams)
    row["HomeTeamWonLast"] = 1 if last_match_winner[teams] == row["Home Team"] else 0
    results.iloc[index] = row
    # Who won this one?
    winner = row["Home Team"] if row["HomeWin"] else row["Visitor Team"]  #  如果主队赢得比赛，主队是本场胜者
    last_match_winner[teams] = winner  # 迭代更新上场比赛的胜者信息
results

Philadelphia 76ers
Charlotte Bobcats
Philadelphia 76ers
Los Angeles Clippers
Portland Trail Blazers
Phoenix Suns
Boston Celtics
New Orleans Pelicans
Cleveland Cavaliers
Phoenix Suns
Charlotte Bobcats
Indiana Pacers
New York Knicks
Golden State Warriors
Cleveland Cavaliers
Houston Rockets
Memphis Grizzlies
Los Angeles Clippers
Utah Jazz
Sacramento Kings
Los Angeles Clippers
Atlanta Hawks
Golden State Warriors
San Antonio Spurs
Los Angeles Clippers
Miami Heat
Oklahoma City Thunder
Charlotte Bobcats
Denver Nuggets
Chicago Bulls
Atlanta Hawks
Memphis Grizzlies
Denver Nuggets
Portland Trail Blazers
Dallas Mavericks
Charlotte Bobcats
Los Angeles Lakers
Golden State Warriors
New Orleans Pelicans
Los Angeles Clippers
Miami Heat
Phoenix Suns
Chicago Bulls
Minnesota Timberwolves
Milwaukee Bucks
Phoenix Suns
Miami Heat
Minnesota Timberwolves
Golden State Warriors
Houston Rockets
Milwaukee Bucks
Orlando Magic
Detroit Pistons
Miami Heat
Memphis Grizzlies
Los Angeles Lakers
Washington Wizards
Chicag

Sacramento Kings
Miami Heat
Dallas Mavericks
Phoenix Suns
Houston Rockets
Oklahoma City Thunder
Toronto Raptors
New Orleans Pelicans
Toronto Raptors
Charlotte Bobcats
Los Angeles Clippers
Washington Wizards
Milwaukee Bucks
Philadelphia 76ers
Atlanta Hawks
Chicago Bulls
Detroit Pistons
Golden State Warriors
Indiana Pacers
Memphis Grizzlies
Minnesota Timberwolves
Oklahoma City Thunder
New York Knicks
Golden State Warriors
Brooklyn Nets
New Orleans Pelicans
Denver Nuggets
Philadelphia 76ers
Phoenix Suns
San Antonio Spurs
Toronto Raptors
Dallas Mavericks
Washington Wizards
Atlanta Hawks
Houston Rockets
Los Angeles Lakers
Phoenix Suns
Minnesota Timberwolves
Golden State Warriors
Miami Heat
New Orleans Pelicans
Washington Wizards
Indiana Pacers
San Antonio Spurs
Toronto Raptors
Los Angeles Clippers
Denver Nuggets
Sacramento Kings
Detroit Pistons
Milwaukee Bucks
Washington Wizards
Cleveland Cavaliers
Houston Rockets
Charlotte Bobcats
Minnesota Timberwolves
New Orleans Pelicans
Oklahoma City T

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes,HomeWin,HomeLastWin,VisitorLastWin,HomeWinStreak,VisitorWinStreak,HomeTeamRanksHigher,HomeTeamWonLast
0,2013-10-29,Box Score,Orlando Magic,87,Indiana Pacers,97,NaN,NaN,True,0,0,0,0,1,0
1,2013-10-29,Box Score,Los Angeles Clippers,103,Los Angeles Lakers,116,NaN,NaN,True,0,0,0,0,0,0
2,2013-10-29,Box Score,Chicago Bulls,95,Miami Heat,107,NaN,NaN,True,0,0,0,0,1,0
3,2013-10-30,Box Score,Brooklyn Nets,94,Cleveland Cavaliers,98,NaN,NaN,True,0,0,0,0,0,0
4,2013-10-30,Box Score,Atlanta Hawks,109,Dallas Mavericks,118,NaN,NaN,True,0,0,0,0,0,0
5,2013-10-30,Box Score,Washington Wizards,102,Detroit Pistons,113,NaN,NaN,True,0,0,0,0,1,0
6,2013-10-30,Box Score,Los Angeles Lakers,94,Golden State Warriors,125,NaN,NaN,True,0,True,0,1,1,0
7,2013-10-30,Box Score,Charlotte Bobcats,83,Houston Rockets,96,NaN,NaN,True,0,0,0,0,1,0
8,2013-10-30,Box Score,Orlando Magic,115,Minnesota Timberwolves,120,OT,NaN,True,0,False,0,0,1,0
9,2013-10-30,Box Score,Indiana Pacers,95,New Orleans Pelicans,90,NaN,NaN,False,0,True,0,1,0,0


In [21]:
X_home_higher =  results[["HomeTeamRanksHigher", "HomeTeamWonLast"]].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_home_higher, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher and which team won the last match")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))


Using whether the home team is ranked higher and which team won the last match
Accuracy: 57.8%


In [74]:
# 为决策树添加球队信息
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encoding = LabelEncoder()  # 把字符串类型转化为整型，其目的是为了把球队与编号一一对应
print(results["Home Team"].values.shape)
# encoding.fit(results["Home Team"].values)
home_teams = encoding.fit_transform(results["Home Team"].values)
visitor_teams = encoding.fit_transform(results["Visitor Team"].values)
X_teams = np.vstack([home_teams, visitor_teams]).T  # 竖直方向拼接的方向拼接，参数先变成横向的行向量，不然无法执行
print((X_teams), X_teams.shape)
onehot = OneHotEncoder()  
X_teams = onehot.fit_transform(X_teams).todense()
print((X_teams), X_teams.shape)

clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_teams, y_true, scoring='accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

(1319,)
[[11 21]
 [13 12]
 [15  4]
 ...
 [15 26]
 [15 26]
 [26 15]] (1319, 2)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (1319, 60)
Accuracy: 59.5%


In [76]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=14)  # 使用随机森林算法，效果有提升
scores = cross_val_score(clf, X_teams, y_true, scoring='accuracy')
print("Using full team labels is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Using full team labels is ranked higher
Accuracy: 60.0%


In [84]:
X_all = np.hstack([X_home_higher, X_teams])  # 把先前计算的主队 是否比对手水平更高和上场比赛的胜者 这两个特征融合 
print(X_all.shape)

(1319, 62)


In [85]:
clf = RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_all, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Using whether the home team is ranked higher
Accuracy: 59.3%


In [86]:
#n_estimators=10, criterion='gini', max_depth=None, 
#min_samples_split=2, min_samples_leaf=1,
#max_features='auto',
#max_leaf_nodes=None, bootstrap=True,
#oob_score=False, n_jobs=1,
#random_state=None, verbose=0, min_density=None, compute_importances=None
parameter_space = {
                   "max_features": [2, 10, 'auto'],
                   "n_estimators": [100,],
                   "criterion": ["gini", "entropy"],
                   "min_samples_leaf": [2, 4, 6],
                   }
clf = RandomForestClassifier(random_state=14)
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_all, y_true)
print("Accuracy: {0:.1f}%".format(grid.best_score_ * 100))
print(grid.best_estimator_)

Accuracy: 64.1%
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=14, verbose=0, warm_start=False)


In [22]:
for exp in range(7,11):
    print('%3d%3s%-12d'%(exp, ' ',10**exp))

  7   10000000    
  8   100000000   
  9   1000000000  
 10   10000000000 


In [28]:
X_teams = np.vstack([np.arange(0,6), np.arange(6,12)])  # 它是垂直（按照行顺序）的把数组（list）给堆叠起来
X_teams 

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11]])

In [32]:
import numpy as np
a=[[1],[2],[3]]  # 列表默认以列排列
b=[[1],[2],[3]]
c=[[1],[2],[3]]
d=[[1],[2],[3]]
print(np.hstack((a,b,c,d)))  #  水平堆叠数组或者list

[[1 1 1 1]
 [2 2 2 2]
 [3 3 3 3]]


In [45]:
print(np.concatenate([np.arange(0,6), np.arange(6,12)]),'\n') # 沿仅有的一个轴拼接
# 沿着垂直0轴的方向拼接，即不改变0轴的大小，横向拼接
print(np.concatenate([np.arange(0,6).reshape(1,-1), np.arange(6,12).reshape(1,-1)]),'\n') 
# 沿着垂直1轴的方向拼接，即不改变1轴的大小，纵向拼接
print(np.concatenate([np.arange(0,6).reshape(1,-1), np.arange(6,12).reshape(1,-1)], axis=1))


[ 0  1  2  3  4  5  6  7  8  9 10 11] 

[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]] 

[[ 0  1  2  3  4  5  6  7  8  9 10 11]]


In [71]:
onehot = OneHotEncoder()
res = onehot.fit_transform(np.array([1,2,3,4,5,6]).reshape(-1,2)).todense()
res


matrix([[1., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 1.]])

In [73]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
para=['a', 'b', 'c']
encoding = LabelEncoder()  # 把字符串类型转化为整型
encoding_result = encoding.fit_transform(para)  # para为字符串列表
print(encoding_result)

[0 1 2]
